This notebook eventually needs to be converted to a script to automate input file processing.  The basic idea is to discretize a geographic area into squares with a given edge length.  Then count the occurances of reports of moving vessels in each of the squares.  This is an imperfect measure of traffic density because AIS reports are not made with uniform frequency.

In [ ]:
import csv
import json
import numpy as np
import numpy.matlib
from collections import defaultdict

I filtered the original data file first using `awk` to get rid of reports from vessels that weren't moving.

```
awk '{ if($5 > 0) { print }}' AIS_2015_01_Zone10.csv >> AIS_2015_01_Zone10_moving.csv 
```

We have data in terms of lat and lon.  We need to convert the desired edge length of the squares into degrees of latitude and longitude.  This relationship is not constant.  The distance between lines of latitude varies slightly as you travel N-S.  The distance between lines of longitude varies significantly as you travel N-S.  The function below takes care of this assuming that the positional data is in WGS-84.

In [ ]:
def edge_len(lat, edge_yds):
    """
    Given a latitude and desired edge length in yards, returns the edge length
    in terms of degrees of latitude.  Assumes a WGS-84 projection.
    
    Args:
        lat: float giving lattitude following the north positive,
             south negative convention.
        edge_yds: float giving the desired lenght of an edge of
                  the bounding square in yards
                  
    Returns: tuple of two floats giving the edge lenght in terms of
             degrees of latitude and degrees of longitude
            
    """
    # We need the latitude in radians
    lat_in_radians = np.deg2rad(lat)
    
    # These came from https://en.wikipedia.org/wiki/Geographic_coordinate_system
    degree_lat_in_meters = 111132.92 - 559.82 * np.cos(2 * lat_in_radians) + 1.175 * np.cos(4 * lat_in_radians) - 0.0023 * np.cos(6 * lat_in_radians)
    degree_lon_in_meters = 111412.84 * np.cos(lat_in_radians) - 93.5 * np.cos(3 * lat_in_radians) + 0.118 * np.cos(5 * lat_in_radians)
    
    # Conversion factor from meters to yards provided by Google
    degree_lat_in_yds = degree_lat_in_meters * 1.09361
    degree_lon_in_yds = degree_lon_in_meters * 1.09361
    
    # Output
    return 1 / degree_lat_in_yds * edge_yds, 1 / degree_lon_in_yds * edge_yds

In [ ]:
class ais_map:
    def __init__(self, mid_lat, mid_lng, edge_yds, sqs_ns, sqs_ew):
        # Establish the length of the squares expressed in degrees of lat and lon
        self.ns_edge, self.ew_edge = edge_len(mid_lat, edge_yds)
        # Compute the northernmost and southernmost latitudes
        self.area_top, self.area_bottom = mid_lat + np.array([1, -1]) * self.ns_edge * sqs_ns / 2
        # Compute the easternmost and westernmost longitudes
        self.area_right, self.area_left = mid_lng + np.array([1, -1]) * self.ew_edge * sqs_ew / 2
        # Discretize latitude
        self.ns_breaks = np.linspace(self.area_top, self.area_bottom, sqs_ns + 1)
        # Discretize longitude
        self.ew_breaks = np.linspace(self.area_left, self.area_right, sqs_ew + 1)
        # I'm building four matricies in numpy to define the spatial domain
        self.ns_matrix_upper = np.matlib.repmat(self.ns_breaks[:sqs_ns], sqs_ew, 1).T
        self.ns_matrix_lower = np.matlib.repmat(self.ns_breaks[1:], sqs_ew, 1).T
        self.ew_matrix_left = np.matlib.repmat(self.ew_breaks[:sqs_ew], sqs_ns, 1)
        self.ew_matrix_right = np.matlib.repmat(self.ew_breaks[1:], sqs_ns, 1)
        # Initialize a numpy matrix to count the instances of vessels in each box
        self.count_matrix = np.zeros([sqs_ns, sqs_ew])
    
    def add_data(self, datafile):
        '''
        Takes a data file name and counts the instances of vessels in each box.
        '''
        with open(datafile, newline='') as csvfile:
            ais_reader = csv.reader(csvfile, delimiter=',')
            # Skip the header
            next(ais_reader)
            count = 0
            for row in ais_reader:
                count += 1
                if count % 100000 == 0: print(count) # Because processing a file takes forever
                LAT = float(row[2])
                LON = float(row[3])
                SOG = float(row[4])
                if (LAT <= self.area_top) & (LAT > self.area_bottom) & (LON >= self.area_left) & (LON < self.area_right):
                    compare_matrix = (LON >= self.ew_matrix_left) * (LON < self.ew_matrix_right) * (LAT <= self.ns_matrix_upper) * (LAT > self.ns_matrix_lower)
                    self.count_matrix += compare_matrix
                    
    def write_json(self):
        '''
        Writes the count matrix to a json file.
        '''
        flat_count = self.count_matrix.flatten()
        flat_ul_corner_lat = self.ns_matrix_upper.flatten()
        flat_ul_corner_lon = self.ew_matrix_left.flatten()
        
        output_dict = {"type":"FeatureCollection","features":[]}

#### An example - SF Bay

In [ ]:
sf_bay = ais_map(37.8272, -122.2913, 100, 960, 760)

In [ ]:
sf_bay.add_data('AIS_2015_01_Zone10_moving.csv')